In [ ]:
#######################################################################################
# Author: Srijan Verma                                                              #
# School of Pharmacy                                                                #
# Sirimulla Research Group [http://www.sirimullaresearchgroup.com/]                 #
# The University of Texas at El Paso, TX, USA                                       #
# Last modified: 19/12/2019                                                         #
# Copyright (c) 2019 Srijan Verma and Sirimulla Research Group, under MIT license   #
#######################################################################################

In [14]:
import pandas as pd
from tqdm import tqdm_notebook
import numpy as np
from glob import glob
import os
import math

In [16]:
for csv in tqdm_notebook(glob('../dataset/cyt_homo_data/sql_data/relation_D/*csv')):

    df = pd.read_csv(csv, index_col=0)

    a = [">", ">=", "<", "<=", "=", "~"]

    c = 0
    drop_index = []
    for i in (range(len(df))):
        if df['activity_comment'][i] == "Active" or df['activity_comment'][i] == "active" or df['activity_comment'][i] == "inactive" or df['activity_comment'][i] == "Not Active" or df['activity_comment'][i] == "Not Active (inhibition < 50% 10 uM and thus dose-response curve not measured)":
            continue
        if pd.isnull(df['standard_value'][i]):
            drop_index.append(i)
        if df['standard_relation'][i] not in a:
            drop_index.append(i)
        if df['standard_units'][i] != 'nM':
            drop_index.append(i)
        else:
            continue


    unique_drop_index = list(set(drop_index))

    df.reset_index(drop=True, inplace=True)
    df.drop(drop_index, axis=0,inplace=True)
    df.reset_index(drop=True, inplace=True)

    negative_log_ten_values = []
    for i in (range(len(df))):

        if df['activity_comment'][i] == "Active" or df['activity_comment'][i] == "active" or df['activity_comment'][i] == "inactive" or df['activity_comment'][i] == "Not Active" or df['activity_comment'][i] == "Not Active (inhibition < 50% 10 uM and thus dose-response curve not measured)":
            negative_log_ten_values.append(-1)# -1 = 'Not required'
            continue

        if df['standard_value'][i] == 0:
            negative_log_ten_values.append(-2)
            #-2 = 'Log10(0) is not defined'
            continue

        else:
            negative_log_ten_values.append((round(((math.log10(df['standard_value'][i]*0.000000001))*(-1)), 3)))
            continue

    df['-log10(M) Values'] = negative_log_ten_values

    labels = []
    unknown_values = []
    drop_index = []

    for i in (range(len(df))):

        if df['activity_comment'][i] == "Active" or df['activity_comment'][i] == "active":
            labels.append('active')
            continue

        if df['activity_comment'][i] == "inactive" or df['activity_comment'][i] == "Not Active" or df['activity_comment'][i] == "Not Active (inhibition < 50% 10 uM and thus dose-response curve not measured)":
            labels.append('inactive')
            continue

        if df['-log10(M) Values'][i] == -2:
            labels.append('Log10(0) is not defined')
            continue

        if df['-log10(M) Values'][i] >= 5.5 and df['standard_relation'][i] in ["<", "<=", "=", "~"]:
            labels.append('active')
            continue

        if df['-log10(M) Values'][i] <=4.5 and df['standard_relation'][i] in [">", ">=", "=", "~"]:
            labels.append('inactive')
            continue

        if df['-log10(M) Values'][i] > 4.5 and df['-log10(M) Values'][i] < 5.5:
            labels.append('indeterminate')
            continue

        else:
            unknown_values.append(i)
            labels.append('Contradictory statements btw Log value and standard_relation')
            continue

    df['Label'] = labels

    for i in (range(len(df))):

        if df['Label'][i] == "Contradictory statements btw Log value and standard_relation" or df['Label'][i] == "Log10(0) is not defined":
            drop_index.append(i)
            continue

    df.reset_index(drop=True, inplace=True)
    df.drop(drop_index, axis=0,inplace=True)
    df.reset_index(drop=True, inplace=True)
    base = os.path.splitext(os.path.basename(csv))[0]
    base = base.replace('__','_')
    df.rename(columns={'smiles':'Smiles'}, inplace=True)
    df.to_csv('../dataset/cyt_homo_data/filtered/relation_D/' + base + '.csv')

In [18]:
min_all_relation_filtered = []
for csv in tqdm_notebook(glob('../dataset/cyt_homo_data/filtered/all_relation/*csv')):

    df = pd.read_csv(csv, index_col=0)
    
    try:
        min_all_relation_filtered.append(min(df['confidence_score']))
        
    except:
        pass

In [26]:
print(max(min_all_relation_filtered))
print(min(min_all_relation_filtered))

9
0


In [27]:
min_relation_D_filtered = []
for csv in tqdm_notebook(glob('../dataset/cyt_homo_data/filtered/relation_D/*csv')):

    df = pd.read_csv(csv, index_col=0)
    
    try:
        min_relation_D_filtered.append(min(df['confidence_score']))
        
    except:
        pass

In [29]:
print(max(min_relation_D_filtered))
print(min(min_relation_D_filtered))

9
5


In [31]:
len(min_relation_D_filtered)

36

In [50]:
df = pd.read_csv('../dataset/cyt_homo_data/filtered/relation_D/Cytochrome_P450_3A.csv')
len(df)

0

In [51]:
count = []
name = []
for csv in tqdm_notebook(glob('../dataset/cyt_homo_data/filtered/relation_D/*csv')):
    df = pd.read_csv(csv, index_col=0)
    count.append(len(df))
    name.append(os.path.splitext(os.path.basename(csv))[0])

In [53]:
df_stats_fp = pd.read_csv('../dataset/cyt_homo_data/fp_stats/fp_stats_223_files.csv',index_col=0)

In [58]:
arr_names = []
drop_index = []
for i in tqdm_notebook(range(len(df_stats_fp))):
    if df_stats_fp['Size'][i] == 28 or df_stats_fp['Size'][i] < 28:
#         print(df_stats_fp['Size'][i])
        drop_index.append(i)
    else:
        arr_names.append(df_stats_fp['file_name'][i])

In [59]:
len(arr_names)

144

In [60]:
arr_names[0]

'rdk_fp-Cytochrome_P450_26A1.npy'

In [62]:
fps = []
enz = []
for i in arr_names:
    a = i.split('-')
    fps.append(a[0])
    enz.append(a[1])

In [64]:
len(set(enz))

24